In [2]:
# Mout Google Drive
# https://towardsdatascience.com/google-drive-google-colab-github-dont-just-read-do-it-5554d5824228
from google.colab import drive
ROOT = "/content/drive"
drive.mount(ROOT)
# %pwd %ls
# run github settings
%run /content/drive/MyDrive/CNNStanford/pytorch/pytorch_sandbox/Colab_Helper.ipynb

Mounted at /content/drive
mkdir: cannot create directory ‘/content/drive/My Drive/CNNStanford/pytorch’: File exists
PROJECT_PATH: /content/drive/My Drive/CNNStanford/pytorch
/content/drive/My Drive/CNNStanford/pytorch/pytorch_sandbox


In [ ]:
MESSAGE = "ex2.2"
!git add .

In [ ]:
!git commit -m "{MESSAGE}"
!git push "{GIT_PATH}"

Implement manual backpropagation without autograd

In [5]:
import torch
import torch.nn.functional as F


# %% Implement the following mini NN
x = torch.randn(1, 6, requires_grad=True)
w = torch.randn(4, 6, requires_grad=True)
y = x @ w.t()  # transpose w, y size (1,4)
y.retain_grad()  # to later compare with autograd
z = F.relu(y)
z.retain_grad()
t = torch.randn(1, 4, requires_grad=True)  # target for mse
L = F.mse_loss(z, t)

# %% Implement backpropagation (without autograd!) to find dL/dw
# Calculate the gradients analytically, then traverse backward using the chain rule
# Go step by step, comparing to Autograd, using w.grad and .retain_grad() and torch.equal
L.backward() # with autograd, to compare to step-by-step

# %% Start from dL/dz
dL_dz = 2/(z.size(0) * z.size(1)) * (z-t)
torch.equal(dL_dz, z.grad)

# %% then dL/dy = dL/dz * dz/dy
dz_dy = ((z > 0)*1 + (z < 0)*0).to(torch.float)  # or (z>0).to(torch.float)
dL_dy = dL_dz * dz_dy
torch.equal(dL_dy, y.grad)

# %% then dL/dw = dL/dy * dy/dw
dy_dw = x
dL_dw = dL_dy.t() * dy_dw  # the transpose on dL_dy, not on x...
torch.equal(dL_dw, w.grad)


RuntimeError: ignored